In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [78]:
Predicted = pd.read_csv("./SpatialDWLS/Outputs/output_DestVISim.csv")
Target=pd.read_csv("./Data/DestVISim/target.csv")

In [79]:
Predicted

,Unnamed: 0,cell_ID,0,1,2,3,4
0,1,0,0.000000,0.543554,0.456446,0.0,0.000000
1,2,1,0.000000,0.669134,0.330866,0.0,0.000000
2,3,2,0.000000,0.646285,0.353715,0.0,0.000000
3,4,3,0.000000,1.000000,0.000000,0.0,0.000000
4,5,4,0.000000,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...
1595,1596,1595,0.000000,0.000000,0.000000,0.0,1.000000
1596,1597,1596,0.300741,0.000000,0.000000,0.0,0.699259
1597,1598,1597,0.275665,0.000000,0.000000,0.0,0.724335
1598,1599,1598,0.423454,0.000000,0.000000,0.0,0.576546


In [80]:
Target

,Unnamed: 0,0,1,2,3,4
0,0,0.109382,0.458548,0.240255,0.117889,0.073926
1,1,0.106379,0.542767,0.192066,0.080376,0.078412
2,2,0.115552,0.584335,0.148478,0.063775,0.087860
3,3,0.134864,0.583714,0.119500,0.059645,0.102278
4,4,0.156835,0.555801,0.104024,0.063044,0.120296
...,...,...,...,...,...,...
1595,1595,0.270532,0.229576,0.079589,0.065887,0.354416
1596,1596,0.331026,0.218634,0.057692,0.046939,0.345709
1597,1597,0.401695,0.196309,0.046337,0.036350,0.319309
1598,1598,0.478614,0.162905,0.039004,0.030636,0.288841


In [81]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="cell_ID", inplace=True)


In [82]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [83]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [84]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [85]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [86]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [87]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [88]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [89]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [90]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [91]:
np.mean(np.nan_to_num(d))

0.8514759819590522

In [92]:
d

array([0.87977541, 0.84693234, 0.87668254, 0.78488766, 0.86910196])

## Jenson Shannon Divergence

In [93]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.34597655960712076

## F1 Score, Precision, Recall

In [94]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [95]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [96]:
d

,Precision,Recall,F1-score
0.100,0.988564,0.586744,0.736407
0.200,0.866375,0.854015,0.860150
0.300,0.672577,0.943094,0.785189
0.010,1.000000,0.376329,0.546859
0.001,1.000000,0.371625,0.541876
